# Python Script to Decode the encrypted email ids

In [2]:
# import pandas to load csv containing emails
import pandas as pd

In [3]:
ids = pd.read_csv("/content/id.csv")
ids

,Email
0,anu.nupurd2lwcm8uY29t@naukri.com
1,samiksha.gYXB0aXRhLmNvbQ==@naukri.com
2,infocGxhdGxhcC5jb20=@naukri.com
3,pbasub3BwaWEtaW50bC5jb20=@naukri.com


In [4]:
import base64         # Used for Base64 decoding
import codecs         # Used for ROT13 decoding
import urllib.parse   # Used for URL decoding
import binascii       # Used for Hex decoding

# Safely decode Base64 strings
def safe_base64_decode(s):
    try:
        # Base64 strings must be a multiple of 4 in length, so we add padding (=) if needed
        padding = '=' * (-len(s) % 4)
        # Decode the string and convert to readable text (UTF-8)
        decoded = base64.b64decode(s + padding, validate=True).decode('utf-8')
        return decoded
    except Exception:
        # Return None if decoding fails
        return None

# Try decoding only the tail (end part) of the local part of email (before '@')
def try_decode_tail(local_part):
    # Loop through each position in the local part
    for i in range(len(local_part)):
        tail = local_part[i:]  # Get tail from current position to end
        decoded = safe_base64_decode(tail)
        if decoded:
            # If decoding works, return the readable local part
            return local_part[:i] + decoded
    # If nothing decodes, return original
    return local_part

# Main function to decode full email
def decode_email(email):
    # If '@' not present, return as is (not a valid email)
    if '@' not in email:
        return email

    # Split into local part and domain part
    local, domain = email.split('@', 1)

    # Try decoding the tail of the local part (left side)
    decoded_local = try_decode_tail(local)

    # Try decoding the entire domain part (right side) using multiple decoding techniques
    domain = attempt_all_decoders(domain)

    # Rebuild the email with decoded parts
    return f"{decoded_local}@{domain}"

# Try different decoding methods to decode the string
def attempt_all_decoders(s):
    # Try Base64, URL, ROT13, and HEX one by one
    for decoder in [try_base64, try_url_decode, try_rot13, try_hex_decode]:
        result = decoder(s)
        # If decoding gives a valid domain (contains a dot), return it
        if result and '.' in result:
            return result
    # If nothing works, return the original string
    return s

# Decoding using Base64
def try_base64(s):
    return safe_base64_decode(s)

# Decoding using URL encoding
def try_url_decode(s):
    try:
        return urllib.parse.unquote(s)
    except:
        return None

# Decoding using ROT13 cipher (used for hiding text by rotating characters)
def try_rot13(s):
    try:
        return codecs.decode(s, 'rot_13')
    except:
        return None

# Decoding using hexadecimal format
def try_hex_decode(s):
    try:
        return bytes.fromhex(s).decode('utf-8')
    except:
        return None

# ==========================
# Example Usage with a list
# ==========================

# Assuming `ids.Email` is a column from a CSV/Excel file (like a pandas DataFrame)
emails = [email for email in ids.Email]  # Fetch email IDs from dataset

# Print results
print("\n DECRYPTED EMAILS:")
for email in emails:
    print(f"Original: {email}")
    print(f"Decoded : {decode_email(email)}\n")



 DECRYPTED EMAILS:
Original: anu.nupurd2lwcm8uY29t@naukri.com
Decoded : anu.nupurwipro.com@naukri.com

Original: samiksha.gYXB0aXRhLmNvbQ==@naukri.com
Decoded : samiksha.gaptita.com@naukri.com

Original: infocGxhdGxhcC5jb20=@naukri.com
Decoded : infoplatlap.com@naukri.com

Original: pbasub3BwaWEtaW50bC5jb20=@naukri.com
Decoded : pbasuoppia-intl.com@naukri.com

